In [ ]:
from pathlib import Path
from io import StringIO
from datetime import datetime, timedelta
import json
import ctypes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import re
import os
import simplekml
from collections import deque

In [4]:
dir_path_parent = Path(r'C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average')

In [ ]:
path_reference = dir_path_parent/ 'reference'   #rtcm資料   EGNSS / TARI .csv     由rtcm_1005csv.py轉換

path_solution = dir_path_parent/ 'solution'   #pos資料   .pos檔 .csv檔 都可   

dir_path_output = dir_path_parent/ 'output'

In [6]:
# 載入DLL檔案
dll_path = 'double-vrs-analyze-main\dll\CoorTrans.dll'  # 將此路徑更改為正確的DLL檔案路徑
dll = ctypes.CDLL(dll_path)

# 設置函式的輸入和輸出類型
dll.pos2twd.restype = None
dll.pos2twd.argtypes = [ctypes.POINTER(
    ctypes.c_double), ctypes.POINTER(ctypes.c_double)]
dll.twd2pos.restype = None
dll.twd2pos.argtypes = [ctypes.POINTER(
    ctypes.c_double), ctypes.POINTER(ctypes.c_double)]

dll.pos2ecef.restype = None
dll.pos2ecef.argtypes = [ctypes.POINTER(
    ctypes.c_double), ctypes.POINTER(ctypes.c_double)]
dll.ecef2pos.restype = None
dll.ecef2pos.argtypes = [ctypes.POINTER(
    ctypes.c_double), ctypes.POINTER(ctypes.c_double)]

dll.twd2ecef.restype = None
dll.twd2ecef.argtypes = [ctypes.POINTER(
    ctypes.c_double), ctypes.POINTER(ctypes.c_double)]
dll.ecef2twd.restype = None
dll.ecef2twd.argtypes = [ctypes.POINTER(
    ctypes.c_double), ctypes.POINTER(ctypes.c_double)]


def pos2twd(pos: np.ndarray) -> np.ndarray:
    pos_array = (ctypes.c_double * len(pos))(*pos)
    twd_array = (ctypes.c_double * len(pos))()
    dll.pos2twd(pos_array, twd_array)
    return np.array(twd_array)


def twd2pos(twd: np.ndarray) -> np.ndarray:
    twd_array = (ctypes.c_double * len(twd))(*twd)
    pos_array = (ctypes.c_double * len(twd))()
    dll.twd2pos(twd_array, pos_array)
    return np.array(pos_array)


def pos2ecef(pos: np.ndarray) -> np.ndarray:
    pos_array = (ctypes.c_double * len(pos))(*pos)
    ecef_array = (ctypes.c_double * len(pos))()
    dll.pos2ecef(pos_array, ecef_array)
    return np.array(ecef_array)


def ecef2pos(ecef: np.ndarray) -> np.ndarray:
    ecef_array = (ctypes.c_double * len(ecef))(*ecef)
    pos_array = (ctypes.c_double * len(ecef))()
    dll.ecef2pos(ecef_array, pos_array)
    return np.array(pos_array)


def twd2ecef(twd: np.ndarray) -> np.ndarray:
    twd_array = (ctypes.c_double * len(twd))(*twd)
    ecef_array = (ctypes.c_double * len(twd))()
    dll.twd2ecef(twd_array, ecef_array)
    return np.array(ecef_array)


def ecef2twd(ecef: np.ndarray) -> np.ndarray:
    ecef_array = (ctypes.c_double * len(ecef))(*ecef)
    twd_array = (ctypes.c_double * len(ecef))()
    dll.ecef2twd(ecef_array, twd_array)
    return np.array(twd_array)

# 定義 Q2Status 函數
def Q2Status(Q):
    return {
        1: 4,
        2: 5,
        4: 2,
        5: 1,
        6: 3
    }.get(Q, 0)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\2815989315.py:2: SyntaxWarning: invalid escape sequence '\d'
  dll_path = 'double-vrs-analyze-main\dll\CoorTrans.dll'  # 將此路徑更改為正確的DLL檔案路徑


In [7]:
# 將 .pos 檔案轉換為 CSV
def convert_pos_to_csv(pos_filepath: str, csv_output_dir: Path):
    
    title = ['GPST', 'Time_of_Week', 'Latitude_deg', 'Longitude_deg', 'Height_m', 'Q', 'ns',
         'sdn_m', 'sde_m', 'sdu_m', 'sdne_m', 'sdeu_m', 'sdun_m', 'Age_s', 'Ratio']
    # 讀取 .pos 檔案
    df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)

    # 檢查資料筆數是否大於 26000
    if df_raw.shape[0] < 26000:
        print(f"{pos_filepath} 資料筆數小於 26000，已跳過儲存。")
        return
    
    # 提取年份和年中天數，並轉換 GPST 和 Time_of_Week 為 DateTime
    pos_filepath_str = str(pos_filepath)
    match = re.search(r'_(\d{4})-(\d{3})', pos_filepath_str)
    if match:
        year, doy = map(int, match.groups())
        base_date = datetime(year, 1, 1) + timedelta(days=doy - 1)
        df_raw['DateTime'] = base_date + pd.to_timedelta(pd.to_numeric(df_raw['Time_of_Week'], errors='coerce'), unit='s')
    else:
        print(f"無法從 {pos_filepath_str} 中提取年份和年中天數。")
        return

    # 計算 E, N, H 和 Status
    df = pd.DataFrame()
    df['DateTime'] = df_raw['DateTime']
    df['Lat'] = df_raw['Latitude_deg']
    df['Lon'] = df_raw['Longitude_deg']
    df['Alt'] = df_raw['Height_m']
    df[['E', 'N', 'H']] = df_raw.apply(
        lambda row: pos2twd([row['Latitude_deg'], row['Longitude_deg'], row['Height_m']]),
        axis=1,
        result_type='expand'
    )
    df['Status'] = df_raw['Q'].apply(Q2Status)

    # 設定 CSV 檔案儲存路徑
    csv_filepath = csv_output_dir / f"{pos_filepath.stem}.csv"
    df.to_csv(csv_filepath, index=False)
    print(f"{csv_filepath} 已儲存完成。")


# 設定 .pos 檔案資料夾路徑和 CSV 輸出資料夾
data_dir_path = Path(r'C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution')
csv_output_dir = Path(r'C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution')
csv_output_dir.mkdir(exist_ok=True)

# 批次處理 .pos 檔案
for file in data_dir_path.glob('**/*.pos'):
    if file.with_suffix(".csv").exists():
        continue

    ts = datetime.now()
    try:
        convert_pos_to_csv(file.absolute(), csv_output_dir)
    except Exception as e:
        print(f"失敗: {e}")
    else:
        print("完成")

C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)


C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS03_2024-159_06-52-06.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS03_2024-160_06-51-45.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS03_2024-161_06-51-48.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS03_2024-162_06-51-53.pos 資料筆數小於 26000，已跳過儲存。
完成


C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_cs

C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS03_2024-164_06-51-27.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS03_2024-165_06-51-47.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS03_2024-166_06-52-32.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS03_2024-167_06-53-28.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS03_2024-168_06-51-48.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS03_2024-169_06-51-08.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS04_2024-159_06-52-06.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS04_2024-160_06-5

C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_cs

...

C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_cs

C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS05_2024-164_06-51-27.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS05_2024-165_06-51-47.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS05_2024-166_06-52-32.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS05_2024-167_06-53-28.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS05_2024-168_06-51-48.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS05_2024-169_06-51-08.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS06_2024-157_06-52-35.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS06_2024-158_06-5

C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_cs

C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS06_2024-162_06-51-53.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS06_2024-164_06-51-27.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS06_2024-165_06-51-47.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS06_2024-166_06-52-32.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS06_2024-167_06-53-28.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS06_2024-168_06-51-48.pos 資料筆數小於 26000，已跳過儲存。
完成
C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\solution\TAIWAN32_TARI_ENGSS_VRS06_2024-169_06-51-08.pos 資料筆數小於 26000，已跳過儲存。
完成


C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_raw = pd.read_csv(pos_filepath, delim_whitespace=True, skiprows=1, names=title)
C:\Users\asus\AppData\Local\Temp\ipykernel_14480\861077435.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_cs

In [8]:
# 計算POS的平均值
def calculate_pos_mean(filepath):
    df = pd.read_csv(filepath)
    mean_e = df['E'].mean()
    mean_n = df['N'].mean()
    mean_h = df['H'].mean()
    return mean_e, mean_n, mean_h

In [9]:
# 計算RTCM的TWD座標
def calculate_rtcm_mean(filepath):
    df = pd.read_csv(filepath)
    ecef_coords = df[['X', 'Y', 'Z']].to_numpy()
    twd_coords = np.array([ecef2twd(ecef) for ecef in ecef_coords])
    mean_e, mean_n, mean_h = twd_coords.mean(axis=0)
    return mean_e, mean_n, mean_h

In [10]:
# 設定路徑
solution_folder = path_solution
reference_folder = path_reference
output_folder = dir_path_output
os.makedirs(output_folder, exist_ok=True)

In [11]:
# 儲存處理結果
result_storage = {
    'EGNSS_VRS03': [],
    'EGNSS_VRS04': [],
    'EGNSS_VRS05': [],
    'EGNSS_VRS06': [],
    'TARI_VRS03': [],
    'TARI_VRS04': [],
    'TARI_VRS05': [],
    'TARI_VRS06': []
}

In [12]:
def classify_files(file_list, pattern):
    classified = {f"{type}_VRS{num}": [] for type in ["EGNSS", "TARI"] for num in ["03", "04", "05", "06"]}
    for file in file_list:
        match = re.search(pattern, file)
        if match:
            file_type = "EGNSS" if "EGNSS" in file else "TARI" if "TARI" in file else None
            vrs = match.group(1)
            if file_type and f"{file_type}_VRS{vrs}" in classified:
                classified[f"{file_type}_VRS{vrs}"].append(file)
    return classified


In [13]:
def classify_files_pos(file_list):
    # 建立分類結構，分別包含 EGNSS 和 TARI 各自的 VRS03~06 類型
    classified = {f"{type}_VRS{num}": [] for type in ["EGNSS", "TARI"] for num in ["03", "04", "05", "06"]}

    for file in file_list:
        # 正則檢測 VRS03~06
        match = re.search(r'VRS(03|04|05|06)', file)
        if not match:
            continue  # 無法匹配的檔案略過

        vrs = match.group(1)  # 提取站點號碼
        # 根據檔案名稱區分 EGNSS 和 TARI
        classified[f"EGNSS_VRS{vrs}"].append(file)
        classified[f"TARI_VRS{vrs}"].append(file)

    return classified


In [ ]:
# 假設 POS 檔案和 RTCM 檔案的分類
solution_files = [file for file in os.listdir(solution_folder) if file.endswith('.csv')]
reference_files = os.listdir(reference_folder)

# 儲存處理結果
result_storage = {
    'EGNSS_VRS03': [],
    'EGNSS_VRS04': [],
    'EGNSS_VRS05': [],
    'EGNSS_VRS06': [],
    'TARI_VRS03': [],
    'TARI_VRS04': [],
    'TARI_VRS05': [],
    'TARI_VRS06': []
}

# 檢查資料夾是否有 .csv 檔案
if not solution_files:
    print(f"{solution_folder} 中沒有找到 .csv 檔案")
    exit()

# 檔案分類
file_pattern = r'VRS(03|04|05|06)'
solution_classified = classify_files_pos(solution_files)
reference_classified = classify_files(reference_files, file_pattern)

# 逐檔處理資料
for key, sol_files in solution_classified.items():
    if not sol_files:
        print(f"{key} 的資料不完整，略過處理")
        continue
    if key not in reference_classified or not reference_classified[key]:
        print(f"{key} 缺少對應的參考檔案")
        continue

    ref_files = reference_classified[key]

    for sol_file in sol_files:
        match = re.search(r'_(\d{4}-\d{3}_\d{2}-\d{2}-\d{2})', sol_file)
        if not match:
            print(f"無法從檔名解析日期和時間：{sol_file}")
            continue

        timestamp = match.group(1)

        # 找對應的參考檔案
        ref_file = next((ref for ref in ref_files if timestamp in ref), None)
        if not ref_file:
            print(f"找不到對應的參考檔案：{sol_file}")
            continue

        # 確認類型（EGNSS 或 TARI）
        ref_type = "EGNSS" if "EGNSS" in ref_file else "TARI"

        #print(f"處理 {ref_type} 的資料：{sol_file} 與 {ref_file}")



        # 計算 POS 平均值
        pos_path = os.path.join(solution_folder, sol_file)
        pos_df = pd.read_csv(pos_path)
        if pos_df.empty:
            print(f"檔案 {sol_file} 沒有資料，略過")
            continue
        pos_mean = pos_df[['E', 'N', 'H']].mean().to_numpy()

        # 計算 RTCM 平均值
        ref_path = os.path.join(reference_folder, ref_file)
        ref_df = pd.read_csv(ref_path)
        if ref_df.empty:
            print(f"檔案 {ref_file} 沒有資料，略過")
            continue
        ecef_coords = ref_df[['X', 'Y', 'Z']].to_numpy()
        twd_coords = np.array([ecef2twd(ecef) for ecef in ecef_coords])
        rtcm_mean = twd_coords.mean(axis=0)
        lat_lon_alt_coords = np.array([ecef2pos(ecef) for ecef in ecef_coords])
        rtcm_lla_mean = lat_lon_alt_coords.mean(axis=0)

        # 計算偏差
        bias = pos_mean - rtcm_mean

        match = re.match(r'_(\d{4})-(\d{3})_(\d{2})-(\d{2})-(\d{2})', timestamp)
        if match:
            year, day_of_year, hour, minute, second = match.groups()
            year = int(year)
            day_of_year = int(day_of_year)

            # 轉換為具體的日期
            date = datetime(year, 1, 1) + timedelta(days=day_of_year-1)
            formatted_date = date.strftime('%m/%d')  # MM/DD 格式

        # 記錄結果
        result_data = {
            "DateTime": formatted_date,
            "pos_E": pos_mean[0], "pos_N": pos_mean[1], "pos_H": pos_mean[2],
            "rtcm_E": rtcm_mean[0], "rtcm_N": rtcm_mean[1], "rtcm_H": rtcm_mean[2],
            "rtcm_lat": rtcm_lla_mean[0], "rtcm_lon": rtcm_lla_mean[1], "rtcm_alt": rtcm_lla_mean[2],
            "BIAS_E": bias[0], "BIAS_N": bias[1], "BIAS_H": bias[2]
        }

        # 儲存到對應的分類
        result_storage[f'{key}'].append(result_data)

EGNSS_VRS04 缺少對應的參考檔案
EGNSS_VRS05 缺少對應的參考檔案
EGNSS_VRS06 缺少對應的參考檔案
找不到對應的參考檔案：TAIWAN32_TARI_ENGSS_VRS05_2024-153_06-53-46.csv
找不到對應的參考檔案：TAIWAN32_TARI_ENGSS_VRS05_2024-154_06-53-57.csv
找不到對應的參考檔案：TAIWAN32_TARI_ENGSS_VRS05_2024-155_06-51-34.csv
找不到對應的參考檔案：TAIWAN32_TARI_ENGSS_VRS05_2024-156_06-52-45.csv
找不到對應的參考檔案：TAIWAN32_TARI_ENGSS_VRS05_2024-157_06-52-35.csv
找不到對應的參考檔案：TAIWAN32_TARI_ENGSS_VRS05_2024-158_06-52-58.csv
找不到對應的參考檔案：TAIWAN32_TARI_ENGSS_VRS05_2024-163_06-51-07.csv
找不到對應的參考檔案：TAIWAN32_TARI_ENGSS_VRS05_2024-169_06-53-54.csv
找不到對應的參考檔案：TAIWAN32_TARI_ENGSS_VRS05_2024-170_06-52-02.csv
找不到對應的參考檔案：TAIWAN32_TARI_ENGSS_VRS05_2024-171_06-52-40.csv
找不到對應的參考檔案：TAIWAN32_TARI_ENGSS_VRS05_2024-172_06-51-42.csv
找不到對應的參考檔案：TAIWAN32_TARI_ENGSS_VRS05_2024-173_06-51-55.csv
TARI_VRS06 缺少對應的參考檔案


In [15]:

# 當所有資料處理完成後，繪製圖表與儲存 CSV
#for key, result_data_list in result_storage.items():
#    if not result_data_list:
#        print(f"{key} 沒有資料，略過處理")
#        continue

    # 將結果轉換為 DataFrame
#    result_df = pd.DataFrame(result_data_list)

    # 儲存 CSV 檔案
#    csv_filename = f"{key}_bias_results.csv"
#    csv_path = os.path.join(output_folder, csv_filename)
#    result_df.to_csv(csv_path, index=False)
#    print(f"結果儲存至：{csv_path}")

    # 繪製並儲存圖表
#    for component in ['E', 'N', 'H']:
#        plt.figure(figsize=(30, 6))
#        plt.scatter(result_df['DateTime'], result_df[f'BIAS_{component}'], alpha=0.6)
#        plt.title(f"{key} BIAS_{component} vs DateTime")
#        plt.xlabel("DateTime (MM/DD)")
#        plt.ylabel(f"BIAS_{component}")
#        plt.grid()
#        plt.xticks(rotation=45)
#        plot_filename = os.path.join(output_folder, f"{key}_bias_{component}_plot.png")
#        plt.savefig(plot_filename)
#        print(f"散布圖儲存至：{plot_filename}")
#        plt.close()

In [16]:
# 當所有資料處理完成後，繪製圖表與儲存 CSV
for key, result_data_list in result_storage.items():
    if not result_data_list:
        print(f"{key} 沒有資料，略過處理")
        continue

    # 將結果轉換為 DataFrame
    result_df = pd.DataFrame(result_data_list)

    # 儲存 CSV 檔案
    csv_filename = f"{key}_bias_results.csv"
    csv_path = os.path.join(output_folder, csv_filename)
    result_df.to_csv(csv_path, index=False)
    print(f"結果儲存至：{csv_path}")

    # 繪製並儲存 E, N, H 散布圖到同一張圖
    fig, axes = plt.subplots(3, 1, figsize=(30, 12), sharex=True)
    components = ['E', 'N', 'H']
    for i, component in enumerate(components):
        ax = axes[i]
        ax.scatter(result_df['DateTime'], result_df[f'BIAS_{component}'], alpha=0.6)
        ax.set_title(f"{key} BIAS_{component} vs DateTime")
        ax.set_ylabel(f"BIAS_{component}")
        ax.grid()
        if i == 2:  # 只有最後一個子圖需要顯示 X 軸標籤
            ax.set_xlabel("DateTime (MM/DD)")
        ax.tick_params(axis='x', rotation=45)

    # 調整子圖間距
    plt.tight_layout()

    # 儲存組合圖表
    plot_filename = os.path.join(output_folder, f"{key}_bias_combined_plot.png")
    plt.savefig(plot_filename)
    print(f"散布圖儲存至：{plot_filename}")
    plt.close()


結果儲存至：C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\EGNSS_VRS03_bias_results.csv
散布圖儲存至：C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\EGNSS_VRS03_bias_combined_plot.png
EGNSS_VRS04 沒有資料，略過處理
EGNSS_VRS05 沒有資料，略過處理
EGNSS_VRS06 沒有資料，略過處理
結果儲存至：C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\TARI_VRS03_bias_results.csv
散布圖儲存至：C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\TARI_VRS03_bias_combined_plot.png
結果儲存至：C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\TARI_VRS04_bias_results.csv
散布圖儲存至：C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\TARI_VRS04_bias_combined_plot.png
TARI_VRS05 沒有資料，略過處理
TARI_VRS06 沒有資料，略過處理


In [17]:
# 儲存 CSV
for key, result_data_list in result_storage.items():
    if not result_data_list:
        print(f"{key} 沒有資料，略過處理")
        continue

    # 將 EGNS 和 TARI 的資料合併到同一個列表中
    combined_results = []

    # 合併 EGNSS 和 TARI 的結果
    for ref_type in ['EGNSS', 'TARI']:
        result_data = result_storage.get(f'{ref_type}_{key}', [])
        for entry in result_data:
            # 在每一筆資料中加入對應的參考類型標註
            entry['Type'] = ref_type  # 標註資料來源類型 (EGNSS 或 TARI)
            combined_results.append(entry)

    # 將合併結果轉換為 DataFrame
    result_df = pd.DataFrame(combined_results)

    # 儲存 CSV 檔案
    csv_filename = f"VRS{key[-2:]}_data.csv"  # 根據 key 生成檔名 (VRS03, VRS04, ...)
    csv_path = os.path.join(output_folder, csv_filename)
    result_df.to_csv(csv_path, index=False)
    print(f"結果儲存至：{csv_path}")


結果儲存至：C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\VRS03_data.csv
EGNSS_VRS04 沒有資料，略過處理
EGNSS_VRS05 沒有資料，略過處理
EGNSS_VRS06 沒有資料，略過處理
結果儲存至：C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\VRS03_data.csv
結果儲存至：C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\VRS04_data.csv
TARI_VRS05 沒有資料，略過處理
TARI_VRS06 沒有資料，略過處理


In [ ]:
# 路徑設定
output_kml_folder = dir_path_output  # 儲存 KML 的資料夾
os.makedirs(output_kml_folder, exist_ok=True)

# 距離閾值 (公尺)
distance_threshold = 10

# 計算兩點的歐幾里得距離
def calculate_distance(point1, point2):
    return np.sqrt((point1['rtcm_E'] - point2['rtcm_E']) ** 2 + 
                   (point1['rtcm_N'] - point2['rtcm_N']) ** 2 + 
                   (point1['rtcm_H'] - point2['rtcm_H']) ** 2)

# 過濾唯一座標點
def filter_unique_points(df, distance_threshold):
    unique_coords = deque()  # 用於儲存唯一座標
    for _, row in df.iterrows():
        current_point = row[['rtcm_E', 'rtcm_N', 'rtcm_H']].to_dict()
        is_unique = all(calculate_distance(current_point, unique_point) >= distance_threshold for unique_point in unique_coords)
        if is_unique:
            unique_coords.append(current_point)
    return df.loc[df.index.isin([i for i in range(len(unique_coords))])]

# 處理每個 key 的資料
for key, result_data_list in result_storage.items():
    if not result_data_list:
        print(f"{key} 沒有資料，略過處理")
        continue

    # 將結果轉為 DataFrame 並選取目標欄位
    result_df = pd.DataFrame(result_data_list)
    target_df = result_df[['DateTime', 'rtcm_lat', 'rtcm_lon', 'rtcm_alt', 'rtcm_E', 'rtcm_N', 'rtcm_H']]


    # 過濾唯一座標點
    unique_df = filter_unique_points(target_df, distance_threshold)

    # 儲存唯一座標到 CSV
    unique_csv_path = os.path.join(output_kml_folder, f"{key}_unique_coordinates.csv")
    unique_df.to_csv(unique_csv_path, index=False)
    print(f"儲存唯一座標至 CSV: {unique_csv_path}")

    # 生成 KML 檔案
    kml = simplekml.Kml()
    for _, row in unique_df.iterrows():
        pnt = kml.newpoint()
        pnt.coords = [(row['rtcm_lat'], row['rtcm_lon'], row['rtcm_alt'])]  # KML 格式: (經度, 緯度, 高度)
        pnt.altitudemode = simplekml.AltitudeMode.relativetoground
        pnt.description = f"Coordinates: {row['rtcm_E']}, {row['rtcm_N']}, {row['rtcm_H']}"
        #pnt.description = f"Coordinates: {row['rtcm_lat']}, {row['rtcm_lon']}, {row['rtcm_alt']}"
    kml_path = os.path.join(output_kml_folder, f"{key}_bias_combined.kml")
    kml.save(kml_path)
    print(f"儲存 KML 檔案至: {kml_path}")

儲存唯一座標至 CSV: C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\EGNSS_VRS03_unique_coordinates.csv
儲存 KML 檔案至: C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\EGNSS_VRS03_bias_combined.kml
EGNSS_VRS04 沒有資料，略過處理
EGNSS_VRS05 沒有資料，略過處理
EGNSS_VRS06 沒有資料，略過處理
儲存唯一座標至 CSV: C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\TARI_VRS03_unique_coordinates.csv
儲存 KML 檔案至: C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\TARI_VRS03_bias_combined.kml
儲存唯一座標至 CSV: C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\TARI_VRS04_unique_coordinates.csv
儲存 KML 檔案至: C:\Users\asus\Desktop\vrs_pos_rtcm_egnss_average\output\TARI_VRS04_bias_combined.kml
TARI_VRS05 沒有資料，略過處理
TARI_VRS06 沒有資料，略過處理
